In [8]:
!pip install transformers
!pip install tiktoken

In [9]:
import torch
import transformers
import sagemaker
from sagemaker.huggingface import HuggingFace
from sagemaker import get_execution_role

role = get_execution_role()

In [10]:
role

'arn:aws:iam::688567266147:role/service-role/AmazonSageMaker-ExecutionRole-20251215T182948'

In [11]:
sagemaker_session = sagemaker.Session()

### Checking versions

In [12]:
import tiktoken
from importlib.metadata import version
import sys
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
# Removed the trailing '|'
print("transformer version:", version("transformers"))
print("python version:", sys.version)

torch version: 2.6.0
tiktoken version: 0.12.0
transformer version: 4.57.1
python version: 3.12.9 | packaged by conda-forge | (main, Feb 14 2025, 08:00:06) [GCC 13.3.0]


### Making The Estimator for Training

In [13]:
huggingface_estimator = HuggingFace(

    entry_point = 'script.py',
    source_dir = './',
    role = role,
    instance_count = 1,
    instance_type = 'ml.g4dn.xlarge',
    transformers_version = '4.49.0',#after encounter error needed to change this ver.
    pytorch_version = '2.5.1',#after encounter error needs to change to match
    output_path = 's3://sagemaker-text-classifier-uci-news-aggregation/output/',
    py_version = 'py311',#needed to change to match
    hyperparameters = {

        'epoch' : 2,
        'train_batch_size' : 4,
        'valid_batch_size' : 2,
        'learning_rate' : 1e-05
    },

    enable_sagemaker_metrics = True ) #it works like docker 

In [14]:
huggingface_estimator.fit() #upon calling this will allocate gpu and cost money we nee dto create script.py so it will fail to run

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2026-01-17-01-50-56-062


2026-01-17 01:50:57 Starting - Starting the training job...
2026-01-17 01:51:12 Starting - Preparing the instances for training...
2026-01-17 01:51:55 Downloading - Downloading the training image..............................
2026-01-17 01:56:42 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 550.163.01
Current installed NVIDIA driver version is 570.195.03
Skipping CUDA compat setup as newer NVIDIA driver is installed
2026-01-17 01:56:50,397 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2026-01-17 01:56:50,417 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2026-01-17 01:56:50,426 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2026-01-17 01:56:50,429 sagemaker_p